In [ ]:
importlib.reload(vp)
importlib.reload(load_clean_lib)
importlib.reload(table_lib)
importlib.reload(vs)
# OTM_str = f"OTM_{round(OTM_moneyness * 100)}%"

OTM = OTMs[1]

df_old = sum_df.copy()
ATM_strategies = [vs.add_put_and_call_sgy, vs.add_delta_put_and_call_sgy, vs.add_straddle_sgy, vs.add_delta_straddle_sgy, vs.add_self_financed_stock_sgy]
for add_sgy in ATM_strategies:
    df_old = add_sgy(df_old)
    
OTM_strategies = [vs.add_put_and_call_sgy, vs.add_delta_put_and_call_sgy, vs.add_strangle_sgy, vs.add_delta_strangle_sgy, vs.add_butterfly_spread_sgy, vs.add_delta_butterfly_spread_sgy]
for add_sgy in OTM_strategies:
    df_old = add_sgy(df_old, OTM)

error = (
        (df_old["30_call_ATM"] - df_orpy["CF_30_call_ATM"]) + 
        (df_old["30_put_ATM"] - df_orpy["CF_30_put_ATM"]) + 
        (df_old["30_call_OTM"] - df_orpy["CF_30_call_OTM_15%"]) + 
        (df_old["30_put_OTM"] - df_orpy["CF_30_put_OTM_15%"]) + 
        (df_old["30_call_D_ATM"] - df_orpy["CF_D_30_call_ATM"]) + 
        (df_old["30_put_D_ATM"] - df_orpy["CF_D_30_put_ATM"]) +
        (df_old["30_call_D_OTM"] - df_orpy["CF_D_30_call_OTM_15%"]) + 
        (df_old["30_put_D_OTM"] - df_orpy["CF_D_30_put_OTM_15%"]) +
        (df_old["30_straddle"] - df_orpy["CF_30_straddle"]) +
        (df_old["30_D_straddle"] - df_orpy["CF_D_30_straddle"]) +
        (df_old["30_strangle"] - df_orpy["CF_30_strangle_15%"]) +
        (df_old["30_D_strangle"] - df_orpy["CF_D_30_strangle_15%"]) +
        (df_old["30_butterfly_spread"] - df_orpy["CF_30_butterfly_spread_15%"]) +
        (df_old["30_D_butterfly_spread"] - df_orpy["CF_D_30_butterfly_spread_15%"]) +
        (df_old["ticker_change_free"] - df_orpy["CF_stock"]) 
         )

plt.plot(df_orpy["date"], error, label = "error")
plt.show()

plt.plot(df_orpy["date"], np.cumsum(error), label = "error")
plt.show()

# Old option strategies

In [ ]:
def add_put_and_call_sgy(df, OTM_moneyness = None):
    if OTM_moneyness == None:
        OTM_str = "ATM"
        OTM_simple = OTM_str
    else:
        OTM_str = f"OTM_{round(OTM_moneyness * 100)}%"
        OTM_simple = "OTM"

    for put_call in ["put", "call"]:
        for low_high in ["low", "high"]:
            df[f"free_{low_high}_{put_call}_{OTM_str}"] = (
                    df[f"{low_high}_{put_call}_{OTM_str}_price_next"] - (1 + df[f"RF"]) * df[f"{low_high}_{put_call}_{OTM_str}_price"]
            ).shift(1)

    # 30 day version
    T1 = df["low days"].shift(1)
    T2 = df["high days"].shift(1)
    for put_call in ["put", "call"]:
        # df[f"30_{put_call}_{OTM_str}"] = T_day_interpolation(T1=T1, T2=T2, r1=df[f"free_low_{put_call}_{OTM_str}"],
        #                                               r2=df[f"free_high_{put_call}_{OTM_str}"])
        df[f"30_{put_call}_{OTM_simple}"] = T_day_interpolation(T1=T1, T2=T2, r1=df[f"free_low_{put_call}_{OTM_str}"],
                                                      r2=df[f"free_high_{put_call}_{OTM_str}"])
    return df


def add_delta_put_and_call_sgy(df, OTM_moneyness = None):
    if OTM_moneyness == None:
        OTM_str = "ATM"
        OTM_simple = OTM_str
    else:
        OTM_str = f"OTM_{round(OTM_moneyness * 100)}%"
        OTM_simple = "OTM"

    for put_call in ["put", "call"]:
        for low_high in ["low", "high"]:
            df[f"free_{low_high}_{put_call}_D_{OTM_str}"] = (
                    df[f"{low_high}_{put_call}_{OTM_str}_price_next"] - (1 + df[f"RF"]) * df[f"{low_high}_{put_call}_{OTM_str}_price"] -
                    df[f"{low_high}_{put_call}_{OTM_str}_delta"] * df[f"close"] * ((df[f"return"] - df[f"RF"]).shift(-1))
            # Delta hedge (self financed)
            ).shift(1)

    # 30 day version
    T1 = df["low days"].shift(1)
    T2 = df["high days"].shift(1)
    for put_call in ["put", "call"]:
        # df[f"30_{put_call}_D_{OTM_str}"] = T_day_interpolation(T1=T1, T2=T2, r1=df[f"free_low_{put_call}_D_{OTM_str}"],
        #                                                 r2=df[f"free_high_{put_call}_D_{OTM_str}"])
        df[f"30_{put_call}_D_{OTM_simple}"] = T_day_interpolation(T1=T1, T2=T2, r1=df[f"free_low_{put_call}_D_{OTM_str}"],
                                                        r2=df[f"free_high_{put_call}_D_{OTM_str}"])
    return df


# def add_straddle_sgy_old(df):
#     for low_high in ["low", "high"]:
#         df[f"free_{low_high}_straddle"] = df[f"free_{low_high}_put_ATM"] + df[f"free_{low_high}_call_ATM"]
#
#     # 30 day version
#     T1 = df["low days"].shift(1)
#     T2 = df["high days"].shift(1)
#     df["30_straddle"] = T_day_interpolation(T1=T1, T2=T2, r1=df[f"free_low_straddle"], r2=df[f"free_high_straddle"])
#     return df

def add_straddle_sgy(df):
    add_strangle_sgy(df) #Passed without 'OTM_moneyness'
    return df

def add_strangle_sgy(df, OTM_moneyness = None):
    if OTM_moneyness == None:
        OTM_str = "ATM"
        sgy_name = "straddle"
        sgy_name_simple = sgy_name
    else:
        OTM_str = f"OTM_{round(OTM_moneyness * 100)}%"
        sgy_name = f"strangle_{round(OTM_moneyness * 100)}%"
        sgy_name_simple = "strangle"

    for low_high in ["low", "high"]:
        df[f"free_{low_high}_{sgy_name}"] = df[f"free_{low_high}_put_{OTM_str}"] + df[f"free_{low_high}_call_{OTM_str}"]

    # 30 day version
    T1 = df["low days"].shift(1)
    T2 = df["high days"].shift(1)
    # df[f"30_{sgy_name}"] = T_day_interpolation(T1=T1, T2=T2, r1=df[f"free_low_{sgy_name}"], r2=df[f"free_high_{sgy_name}"])
    df[f"30_{sgy_name_simple}"] = T_day_interpolation(T1=T1, T2=T2, r1=df[f"free_low_{sgy_name}"], r2=df[f"free_high_{sgy_name}"])
    return df


def add_butterfly_spread_sgy(df, OTM_moneyness):
    ATM_str = f"ATM"
    OTM_str = f"OTM_{round(OTM_moneyness * 100)}%"
    sgy_name = f"butterfly_spread_{round(OTM_moneyness * 100)}%"
    sgy_name_simple = "butterfly_spread"

    for low_high in ["low", "high"]:
        ATM_payoff = df[f"free_{low_high}_put_{ATM_str}"] + df[f"free_{low_high}_call_{ATM_str}"]
        OTM_payoff = df[f"free_{low_high}_put_{OTM_str}"] + df[f"free_{low_high}_call_{OTM_str}"]
        df[f"free_{low_high}_{sgy_name}"] = ATM_payoff - OTM_payoff

    # 30 day version
    T1 = df["low days"].shift(1)
    T2 = df["high days"].shift(1)
    # df[f"30_{sgy_name}"] = T_day_interpolation(T1=T1, T2=T2, r1=df[f"free_low_{sgy_name}"], r2=df[f"free_high_{sgy_name}"])
    df[f"30_{sgy_name_simple}"] = T_day_interpolation(T1=T1, T2=T2, r1=df[f"free_low_{sgy_name}"], r2=df[f"free_high_{sgy_name}"])
    return df

def add_delta_butterfly_spread_sgy(df, OTM_moneyness):
    ATM_str = f"D_ATM"
    OTM_str = f"D_OTM_{round(OTM_moneyness * 100)}%"
    sgy_name = f"D_butterfly_spread_{round(OTM_moneyness * 100)}%"
    sgy_name_simple = ("D_butterfly_spread")

    for low_high in ["low", "high"]:
        ATM_payoff = df[f"free_{low_high}_put_{ATM_str}"] + df[f"free_{low_high}_call_{ATM_str}"] # buy straddle
        OTM_payoff = df[f"free_{low_high}_put_{OTM_str}"] + df[f"free_{low_high}_call_{OTM_str}"] # sell strangle
        df[f"free_{low_high}_{sgy_name}"] = ATM_payoff - OTM_payoff

    # 30 day version
    T1 = df["low days"].shift(1)
    T2 = df["high days"].shift(1)
    # df[f"30_{sgy_name}"] = T_day_interpolation(T1=T1, T2=T2, r1=df[f"free_low_{sgy_name}"], r2=df[f"free_high_{sgy_name}"])
    df[f"30_{sgy_name_simple}"] = T_day_interpolation(T1=T1, T2=T2, r1=df[f"free_low_{sgy_name}"], r2=df[f"free_high_{sgy_name}"])
    return df


# def add_delta_straddle_sgy_old(df):
#     for low_high in ["low", "high"]:
#         df[f"free_D_{low_high}_straddle"] = df[f"free_D_{low_high}_put"] + df[f"free_D_{low_high}_call"]
#
#     # 30 day version
#     T1 = df["low days"].shift(1)
#     T2 = df["high days"].shift(1)
#     df["30_D_straddle"] = T_day_interpolation(T1=T1, T2=T2, r1=df[f"free_D_low_straddle"],
#                                                  r2=df[f"free_D_high_straddle"])
#     return df

def add_delta_strangle_sgy(df, OTM_moneyness = None):
    if OTM_moneyness == None:
        OTM_str = "D_ATM"
        sgy_name = "D_straddle"
        sgy_name_simple = sgy_name
    else:
        OTM_str = f"D_OTM_{round(OTM_moneyness * 100)}%"
        sgy_name = f"D_strangle_{round(OTM_moneyness * 100)}%"
        sgy_name_simple = "D_strangle"

    for low_high in ["low", "high"]:
        df[f"free_{low_high}_{sgy_name}"] = df[f"free_{low_high}_put_{OTM_str}"] + df[f"free_{low_high}_call_{OTM_str}"]

    # 30 day version
    T1 = df["low days"].shift(1)
    T2 = df["high days"].shift(1)
    # df[f"30_{sgy_name}"] = T_day_interpolation(T1=T1, T2=T2, r1=df[f"free_low_{sgy_name}"], r2=df[f"free_high_{sgy_name}"])
    df[f"30_{sgy_name_simple}"] = T_day_interpolation(T1=T1, T2=T2, r1=df[f"free_low_{sgy_name}"], r2=df[f"free_high_{sgy_name}"])
    return df

def add_delta_straddle_sgy(df):
    add_delta_strangle_sgy(df) #Passed without 'OTM_moneyness'
    return df

def add_self_financed_stock_sgy(df):
    for ticker in df['ticker'].unique():
        ticker_mask = df['ticker'] == ticker
        df.loc[ticker_mask, 'ticker_change_free'] = (
                df.loc[ticker_mask, 'close'] -
                (df.loc[ticker_mask, 'close'] * (1 + df.loc[ticker_mask, 'RF'])).shift(1)
        )
    return df

# more old option strategies (these failed)

In [ ]:

import pandas as pd
import numpy as np

import load_clean_lib
from global_settings import *
from vol_strat_lib import T_day_interpolation



def D_PC_option_prices(df, OTMs):
    ATM_str = "ATM"
    OTM_str = [f"OTM_{round(OTM * 100)}%" for OTM in OTMs]
    moneynesses = [ATM_str] + OTM_str

    T1 = df["low days"]
    T2 = df["high days"]

    for moneyness in moneynesses:
        for put_call in ["put", "call"]:
            for low_high in ["low", "high"]:
                df[f"D_{low_high}_{put_call}_{moneyness}_price"] = (
                    df[f"{low_high}_{put_call}_{moneyness}_price"]
                )

                df[f"D_{low_high}_{put_call}_{moneyness}_price_next"] = (
                    df[f"{low_high}_{put_call}_{moneyness}_price_next"] -
                    df[f"{low_high}_{put_call}_{moneyness}_delta"] * df[f"close"] * (df["return"]-df["RF"]).shift(-1)
                )

            for time in ["", "_next"]:
                for D_str in ["D_", ""]:
                    low_price_time = df[f"{D_str}low_{put_call}_{moneyness}_price{time}"]
                    high_price_time = df[f"{D_str}high_{put_call}_{moneyness}_price{time}"]
                    df[f"{D_str}30_{put_call}_{moneyness}_price{time}"] = T_day_interpolation(T1=T1, T2=T2, r1=low_price_time, r2=high_price_time)
    return df
# df[f"D_{low_high}_{put_call}_{moneyness}_price"] = (
#     df[f"{low_high}_{put_call}_{moneyness}_price"] -
#     df[f"{low_high}_{put_call}_{moneyness}_delta"] * df[f"close"]
# )
# df[f"D_{low_high}_{put_call}_{moneyness}_price_next"] = (
#     df[f"{low_high}_{put_call}_{moneyness}_price_next"] -
#     df[f"{low_high}_{put_call}_{moneyness}_delta"] * df[f"close"] * (1+df["return"].shift(-1))
# )


def add_put_and_call_sgy(df, OTMs):
    ATM_str = "ATM"
    OTM_str = [f"OTM_{round(OTM * 100)}%" for OTM in OTMs]
    moneynesses = [ATM_str] + OTM_str

    for put_call in ["put", "call"]:
        for D_str in ["D_", ""]:
            for moneyness in moneynesses:
                for low_high in ["low", "high", "30"]:
                    next_price = df[f"{D_str}{low_high}_{put_call}_{moneyness}_price_next"]
                    current_price = df[f"{D_str}{low_high}_{put_call}_{moneyness}_price"]

                    CF = (next_price - (1+df["RF"]) * current_price).shift(1)
                    ret = CF / current_price.shift(1) #((next_price - (1+df["RF"]) * current_price)/current_price).shift(1)

                    df[f"CF_{D_str}{low_high}_{put_call}_{moneyness}"] = CF
                    df[f"r_{D_str}{low_high}_{put_call}_{moneyness}"] = ret
    return df


    # T1 = df["low days"].shift(1)
    # T2 = df["high days"].shift(1)


# # 30 day version
# df[f"CF_30_{D_str}{put_call}_{moneyness}"] = T_day_interpolation(T1=T1, T2=T2, r1=df_tmp[f"CF_low"], r2=df_tmp[f"CF_high"])
#
# price_30 = T_day_interpolation(T1=T1, T2=T2, r1=df_tmp[f"price_low"], r2=df_tmp[f"price_high"])
# df[f"r_30_{D_str}{put_call}_{moneyness}"]  = df[f"CF_30_{D_str}{put_call}_{moneyness}"] / price_30


# df_tmp[f"CF_{D_str}{low_high}_{put_call}_{moneyness}"] = (next_price - (1+df["RF"]) * current_price).shift(1)
# df[f"r_{D_str}{low_high}_{put_call}_{moneyness}"] = ((next_price - (1+df["RF"]) * current_price) / current_price).shift(1)

# low = df[f"CF_{D_str}low_{put_call}_{moneyness}"]
# high = df[f"CF_{D_str}high_{put_call}_{moneyness}"]


def add_straddle_strangle_sgy(df, OTMs):
    ATM_str = "ATM"
    OTM_str = [f"OTM_{round(OTM * 100)}%" for OTM in OTMs]

    moneynesses = [ATM_str] + OTM_str
    sgy_names = ["straddle"] + [f"strangle_{round(OTM * 100)}%" for OTM in OTMs]

    def calculate_price(low_high, D_str, moneyness, next_str):
        ATM_price = df[f"{D_str}{low_high}_put_{moneyness}_price{next_str}"] + df[f"{D_str}{low_high}_call_{ATM_str}_price{next_str}"]
        return ATM_price

    for D_str, delta in zip(["D_", ""], [True, False]):
        for moneyness, sgy_name in zip(moneynesses, sgy_names):
            for low_high in ["low", "high", "30"]:
                next_price = calculate_price(low_high, D_str, moneyness, next_str="_next")
                current_price = calculate_price(low_high, D_str, moneyness, next_str="")

                df[f"CF_{D_str}{sgy_name}_{low_high}"] = (next_price - (1 + df["RF"]) * current_price).shift(1)
                df[f"r_{D_str}{sgy_name}_{low_high}"] = ((next_price - (1 + df["RF"]) * current_price) / current_price).shift(1)
    return df


# low = df_tmp[f"CF_{D_str}{sgy_name}_low"]
# high = df_tmp[f"CF_{D_str}{sgy_name}_high"]
# df[f"CF_30_{D_str}{sgy_name}"] = T_day_interpolation(T1=T1, T2=T2, r1=low, r2=high)
#
# low = df_tmp["low"]
# high = df_tmp["high"]
# current_price_30 = T_day_interpolation(T1=T1, T2=T2, r1=low, r2=high)
# df[f"r_30_{D_str}{sgy_name}"] = df[f"CF_30_{D_str}{sgy_name}"] / current_price_30


# def add_straddle_strangle_sgy(df, OTMs):
#     ATM_str = "ATM"
#     OTM_str = [f"OTM_{round(OTM * 100)}%" for OTM in OTMs]
#
#     moneynesses = [ATM_str] + OTM_str
#     sgy_names = ["straddle"] + [f"strangle_{round(OTM * 100)}%" for OTM in OTMs]
#
#     T1 = df["low days"].shift(1)
#     T2 = df["high days"].shift(1)
#
#     def calculate_price(low_high, D_str, moneyness, next_str):
#         ATM_price = df[f"{D_str}{low_high}_put_{moneyness}_price{next_str}"] + df[f"{D_str}{low_high}_call_{ATM_str}_price{next_str}"]
#         return ATM_price
#
#
#     for D_str, delta in zip(["D_", ""], [True, False]):
#         for moneyness, sgy_name in zip(moneynesses, sgy_names):
#             df_tmp = pd.DataFrame()
#             for low_high in ["low", "high"]:
#                 # for CF_r in ["CF", "r"]:
#                 #     df[f"{CF_r}_{D_str}{sgy_name}{low_high}"] = (df[f"{CF_r}_{D_str}{low_high}_put_{moneyness}"] + df[f"{CF_r}_{D_str}{low_high}_call_{moneyness}"]) / 2 #todo: talk with axel about this average (div. by 2)
#
#                 next_price = calculate_price(low_high, D_str, moneyness, next_str="_next")
#                 current_price = calculate_price(low_high, D_str, moneyness, next_str="")
#                 df_tmp[low_high] = current_price
#
#                 df_tmp[f"CF_{D_str}{sgy_name}_{low_high}"] = (next_price - (1 + df["RF"]) * current_price).shift(1)
#                 # df_tmp[f"r_{D_str}{sgy_name}_{low_high}"] = ((next_price - (1 + df["RF"]) * current_price) / current_price).shift(1)
#
#             low  = df_tmp[f"CF_{D_str}{sgy_name}_low"]
#             high = df_tmp[f"CF_{D_str}{sgy_name}_high"]
#             df[f"CF_30_{D_str}{sgy_name}"] = T_day_interpolation(T1=T1, T2=T2, r1=low, r2=high)
#
#             low  = df_tmp["low"]
#             high = df_tmp["high"]
#             current_price_30 = T_day_interpolation(T1=T1, T2=T2, r1=low, r2=high)
#             df[f"r_30_{D_str}{sgy_name}"] = df[f"CF_30_{D_str}{sgy_name}"] / current_price_30
#     return df



# def add_butterfly_spread_sgy(df, OTMs):
#     ATM_str = "ATM"
#     OTM_strs = [f"OTM_{round(OTM * 100)}%" for OTM in OTMs]
#     sgy_names = [f"butterfly_spread_{round(OTM * 100)}%" for OTM in OTMs]
#
#     T1 = df["low days"].shift(1)
#     T2 = df["high days"].shift(1)
#
#     # Function to calculate the price difference between ATM and OTM for a given scenario
#     def calculate_price(low_high, D_str, OTM_str, next_str):
#         ATM_price = df[f"{D_str}{low_high}_put_{ATM_str}_price{next_str}"] + df[f"{D_str}{low_high}_call_{ATM_str}_price{next_str}"]
#         OTM_price = df[f"{D_str}{low_high}_put_{OTM_str}_price{next_str}"] + df[f"{D_str}{low_high}_call_{OTM_str}_price{next_str}"]
#         return ATM_price - OTM_price
#
#     for D_str, delta in zip(["D_", ""], [True, False]):
#         for OTM_str, sgy_name in zip(OTM_strs, sgy_names):
#             for low_high in ["low", "high"]:
#                 next_price = calculate_price(low_high, D_str, OTM_str, next_str = "_next")
#                 current_price = calculate_price(low_high, D_str, OTM_str, next_str = "")
#
#                 df[f"CF_{D_str}{sgy_name}_{low_high}"] = (next_price - (1+df["RF"]) * current_price).shift(1)
#                 df[f"r_{D_str}{sgy_name}_{low_high}"] = ((next_price - (1+df["RF"]) * current_price) / current_price).shift(1)
#
#             low_CF = df[f"CF_{D_str}{sgy_name}_{const_str}low"]
#             high_CF = df[f"CF_{D_str}{sgy_name}_{const_str}high"]
#             df[f"CF_30_{D_str}{sgy_name}_{const_str}"] = T_day_interpolation(T1=T1, T2=T2, r1=low_CF, r2=high_CF)
#     return df


In [ ]:

import pandas as pd
import numpy as np

import load_clean_lib
from global_settings import *
from vol_strat_lib import T_day_interpolation

def D_PC_option_prices(df, OTMs):
    ATM_str = "ATM"
    OTM_str = [f"OTM_{round(OTM * 100)}%" for OTM in OTMs]
    moneynesses = [ATM_str] + OTM_str

    for moneyness in moneynesses:
        for low_high in ["low", "high"]:
            for put_call in ["put", "call"]:
                df[f"D_{low_high}_{put_call}_{moneyness}_price"] = (
                    df[f"{low_high}_{put_call}_{moneyness}_price"] +
                    df[f"{low_high}_{put_call}_{moneyness}_delta"] * df[f"close"]
                )
                df[f"D_{low_high}_{put_call}_{moneyness}_price_next"] = (
                    df[f"{low_high}_{put_call}_{moneyness}_price_next"] +
                    df[f"{low_high}_{put_call}_{moneyness}_delta"] * df[f"close"] * df["return"]
                )
    return df
# D_low_put_ATM_price_next
def add_put_and_call_sgy(df, OTMs):
    ATM_str = "ATM"
    OTM_str = [f"OTM_{round(OTM * 100)}%" for OTM in OTMs]
    moneynesses = [ATM_str] + OTM_str

    T1 = df["low days"].shift(1)
    T2 = df["high days"].shift(1)

    for put_call in ["put", "call"]:
        for const_str, const_cost in zip(["const_",""],[True, False]):
            for D_str in ["D_", ""]:
                for moneyness in moneynesses:
                    for low_high in ["low", "high"]:
                        option_change = df[f"{D_str}{low_high}_{put_call}_{moneyness}_price_next"] - (1 + df[f"RF"]) * df[f"{D_str}{low_high}_{put_call}_{moneyness}_price"]
                        if const_cost:
                            option_price = (1 + df[f"RF"]) * df[f"{low_high}_{put_call}_{moneyness}_price"]
                        else:
                            option_price = 1

                        df[f"CF_{const_str}{D_str}{low_high}_{put_call}_{moneyness}"] = (
                            option_change / option_price
                        ).shift(1)

                    # 30 day version
                    low_CF = df[f"CF_{const_str}{D_str}low_{put_call}_{moneyness}"]
                    high_CF = df[f"CF_{const_str}{D_str}high_{put_call}_{moneyness}"]
                    df[f"CF_30_{const_str}{D_str}{put_call}_{moneyness}"] = T_day_interpolation(T1=T1, T2=T2, r1=low_CF, r2=high_CF)
    return df

def add_straddle_strangle_sgy(df, OTMs):
    ATM_str = "ATM"
    OTM_str = [f"OTM_{round(OTM * 100)}%" for OTM in OTMs]

    moneynesses = [ATM_str] + OTM_str
    sgy_names = ["straddle"] + [f"strangle_{round(OTM * 100)}%" for OTM in OTMs]

    T1 = df["low days"].shift(1)
    T2 = df["high days"].shift(1)

    for D_str, delta in zip(["D_", ""], [True, False]):
        for moneyness, sgy_name in zip(moneynesses, sgy_names):
            for low_high in ["low", "high"]:
                sgy_CF = df[f"CF_{D_str}{low_high}_put_{moneyness}"] + df[f"CF_{D_str}{low_high}_call_{moneyness}"]

                for const_str, const_cost in zip(["const_", ""], [True, False]):
                    if const_cost:
                        price = df[f"{D_str}{low_high}_put_{moneyness}_price"] + df[f"{D_str}{low_high}_call_{moneyness}_price"]
                    else:
                        price = 1
                    df[f"CF_{sgy_name}_{const_str}{D_str}{low_high}"] = sgy_CF / price
            for const_str in ["const_", ""]:
                low_CF  = df[f"CF_{sgy_name}_{const_str}{D_str}low"]
                high_CF = df[f"CF_{sgy_name}_{const_str}{D_str}high"]
                df[f"CF_30_{sgy_name}_{const_str}{D_str}"] = T_day_interpolation(T1=T1, T2=T2, r1=low_CF, r2=high_CF)
    return df

def add_butterfly_spread_sgy(df, OTMs):
    ATM_str = "ATM"
    OTM_strs = [f"OTM_{round(OTM * 100)}%" for OTM in OTMs]
    sgy_names = [f"butterfly_spread_{round(OTM * 100)}%" for OTM in OTMs]

    T1 = df["low days"].shift(1)
    T2 = df["high days"].shift(1)

    # Function to calculate the price difference between ATM and OTM for a given scenario
    def calculate_price(low_high, D_str, OTM_str, next_str):
        ATM_price = df[f"{D_str}{low_high}_put_{ATM_str}_price{next_str}"] + df[f"{D_str}{low_high}_call_{ATM_str}_price{next_str}"]
        OTM_price = df[f"{D_str}{low_high}_put_{OTM_str}_price{next_str}"] + df[f"{D_str}{low_high}_call_{OTM_str}_price{next_str}"]
        return ATM_price - OTM_price

    for D_str, delta in zip(["D_", ""], [True, False]):
        for OTM_str, sgy_name in zip(OTM_strs, sgy_names):
            for low_high in ["low", "high"]:
                sgy_CF = calculate_price(low_high, D_str, OTM_str, next_str = "_next") - calculate_price(low_high, D_str, OTM_str, next_str = "")
                for const_str, const_cost in zip(["const_", ""], [True, False]):
                    if const_cost:
                        price = calculate_price(low_high, D_str, OTM_str, next_str = "")
                    else:
                        price = 1
                    df[f"CF_{sgy_name}_{const_str}{D_str}{low_high}"] = sgy_CF / price

            low_CF = df[f"CF_{sgy_name}_{const_str}{D_str}low"]
            high_CF = df[f"CF_{sgy_name}_{const_str}{D_str}high"]
            df[f"CF_30_{sgy_name}_{const_str}{D_str}"] = T_day_interpolation(T1=T1, T2=T2, r1=low_CF, r2=high_CF)
    return df

def add_self_financed_stock_sgy(df, _ = None):
    for ticker in df['ticker'].unique():
        for const_str, const_cost in zip(["_const", ""], [True, False]):
            ticker_mask = df['ticker'] == ticker
            if const_cost:
                price = df.loc[ticker_mask, 'close'].shift(1) * (1 + df.loc[ticker_mask, 'RF'])
            else:
                price = 1
            CF_unscaled = df.loc[ticker_mask, 'close'] - df.loc[ticker_mask, 'close'].shift(1) * (1 + df.loc[ticker_mask, 'RF'])
            df.loc[ticker_mask, f'CF_ticker{const_str}'] = CF_unscaled / price
    return df



# def add_put_and_call_sgy(df, OTM_moneyness = None):
#     if OTM_moneyness == None:
#         OTM_str = "ATM"
#         OTM_simple = OTM_str
#     else:
#         OTM_str = f"OTM_{round(OTM_moneyness * 100)}%"
#         OTM_simple = "OTM"
#
#         for low_high in ["low", "high"]:
#             df[f"const_free_{low_high}_{put_call}_{OTM_str}"] = (
#                     df[f"free_{low_high}_{put_call}_{OTM_str}"] / (1 + df[f"RF"]) * df[f"{low_high}_{put_call}_{OTM_str}_price"]
#             ).shift(1)    for put_call in ["put", "call"]:
#
#
#     # 30 day version
#     T1 = df["low days"].shift(1)
#     T2 = df["high days"].shift(1)
#     for put_call in ["put", "call"]:
#         df[f"const_30_{put_call}_{OTM_simple}"] = (df[f"30_{put_call}_{OTM_simple}"] /
#             T_day_interpolation(T1=T1, T2=T2, r1=df[f"low_{put_call}_{OTM_str}_price"], r2=df[f"high_{put_call}_{OTM_str}_price"]))
#     return df
#
#
# def add_delta_put_and_call_sgy(df, OTM_moneyness = None):
#     if OTM_moneyness == None:
#         OTM_str = "ATM"
#         OTM_simple = OTM_str
#     else:
#         OTM_str = f"OTM_{round(OTM_moneyness * 100)}%"
#         OTM_simple = "OTM"
#
#     for put_call in ["put", "call"]:
#         for low_high in ["low", "high"]:
#             option_return = df[f"{low_high}_{put_call}_{OTM_str}_price_next"] - (1 + df[f"RF"]) * df[f"{low_high}_{put_call}_{OTM_str}_price"]
#             delta_return = df[f"{low_high}_{put_call}_{OTM_str}_delta"] * df[f"close"] * (df[f"return"].shift(-1) - df[f"RF"])
#             df[f"free_{low_high}_{put_call}_D_{OTM_str}"] = (
#                     df[f"free_{low_high}_{put_call}_D_{OTM_str}"] /
#                     ((1 + df[f"RF"]) * (df[f"{low_high}_{put_call}_{OTM_str}_price"] + df[f"{low_high}_{put_call}_{OTM_str}_delta"] * df[f"close"]))
#                                                              ).shift(1)
#
#     # 30 day version
#     T1 = df["low days"].shift(1)
#     T2 = df["high days"].shift(1)
#     for put_call in ["put", "call"]:
#         price_low  = ((1 + df[f"RF"]) * df[f"low_{put_call}_{OTM_str}_price"] + df[f"low_{put_call}_{OTM_str}_delta"] * df[f"close"])
#         price_high = ((1 + df[f"RF"]) * df[f"high_{put_call}_{OTM_str}_price"] + df[f"high_{put_call}_{OTM_str}_delta"] * df[f"close"])
#         df[f"const_30_{put_call}_D_{OTM_simple}"] = df[f"30_{put_call}_D_{OTM_simple}"] / T_day_interpolation(T1=T1, T2=T2, r1=price_low, r2=price_high)
#     return df
#
#
# # def add_straddle_sgy_old(df):
# #     for low_high in ["low", "high"]:
# #         df[f"free_{low_high}_straddle"] = df[f"free_{low_high}_put_ATM"] + df[f"free_{low_high}_call_ATM"]
# #
# #     # 30 day version
# #     T1 = df["low days"].shift(1)
# #     T2 = df["high days"].shift(1)
# #     df["30_straddle"] = T_day_interpolation(T1=T1, T2=T2, r1=df[f"free_low_straddle"], r2=df[f"free_high_straddle"])
# #     return df
#
# def add_straddle_sgy(df):
#     add_strangle_sgy(df) #Passed without 'OTM_moneyness'
#     return df
#
# def add_strangle_sgy(df, OTM_moneyness = None, delta_version = False):
#     if OTM_moneyness == None:
#         OTM_str = "ATM"
#         sgy_name = "straddle"
#         sgy_name_simple = sgy_name
#     else:
#         OTM_str = f"OTM_{round(OTM_moneyness * 100)}%"
#         sgy_name = f"strangle_{round(OTM_moneyness * 100)}%"
#         sgy_name_simple = "strangle"
#
#     if delta_version:
#         OTM_str = "D_" + OTM_str
#         sgy_name = "D_" + sgy_name
#         sgy_name_simple = "D_" + sgy_name_simple
#
#
#
#     for low_high in ["low", "high"]:
#         put_price = ((1 + df[f"RF"]) * (df[f"{low_high}_put_{OTM_str}_price"] + delta_version * df[f"{low_high}_put_{OTM_str}_delta"] * df[f"close"]))
#         call_price = ((1 + df[f"RF"]) * (df[f"{low_high}_call_{OTM_str}_price"] + delta_version * df[f"{low_high}_call_{OTM_str}_delta"] * df[f"close"]))
#
#         df[f"const_free_{low_high}_{sgy_name}"] = df[f"free_{low_high}_{sgy_name}"] / (put_price + call_price)
#
#     # 30 day version
#     T1 = df["low days"].shift(1)
#     T2 = df["high days"].shift(1)
#
#     price_low  = df[f"RF"] * (df[f"low_put_{OTM_str}_price"]  + df[f"low_call_{OTM_str}_price"])
#     price_high = df[f"RF"] * (df[f"high_put_{OTM_str}_price"] + df[f"high_call_{OTM_str}_price"])
#     df[f"const_30_{sgy_name_simple}"] = df[f"30_{sgy_name_simple}"] / T_day_interpolation(T1=T1, T2=T2, r1=price_low, r2=price_high)
#     return df
#
#
# def add_butterfly_spread_sgy(df, OTM_moneyness, delta_version = False):
#     ATM_str = f"ATM"
#     OTM_str = f"OTM_{round(OTM_moneyness * 100)}%"
#     sgy_name = f"butterfly_spread_{round(OTM_moneyness * 100)}%"
#     sgy_name_simple = "butterfly_spread"
#
#     if delta_version:
#         ATM_str = "D_" + ATM_str
#         OTM_str = "D_" + OTM_str
#         sgy_name = "D_" + sgy_name
#         sgy_name_simple = "D_" + sgy_name_simple
#
#     # Function to calculate the price difference between ATM and OTM for a given scenario
#     def calculate_price(low_high):
#         ATM_price = df[f"{low_high}_put_{ATM_str}_price"] + df[f"{low_high}_call_{ATM_str}_price"]
#         OTM_price = df[f"{low_high}_put_{OTM_str}_price"] + df[f"{low_high}_call_{OTM_str}_price"]
#         return ATM_price - OTM_price
#
#
#     for low_high in ["low", "high"]:
#         df[f"const_free_{low_high}_{sgy_name}"] = df[f"free_{low_high}_{sgy_name}"] / calculate_price(low_high)
#
#     # 30 day version
#     T1 = df["low days"].shift(1)
#     T2 = df["high days"].shift(1)
#
#     df[f"const_30_{sgy_name_simple}"] = df[f"30_{sgy_name_simple}"] / T_day_interpolation(T1=T1, T2=T2, r1=calculate_price('low'), r2=calculate_price('high'))
#     return df
#
#
# def add_delta_butterfly_spread_sgy(df, OTM_moneyness):
#     return add_butterfly_spread_sgy(df, OTM_moneyness, delta_version=True)
#
# def add_delta_strangle_sgy(df, OTM_moneyness = None):
#     return add_strangle_sgy(df, OTM_moneyness, delta_version = True)
#
#
# # def add_delta_butterfly_spread_sgy(df, OTM_moneyness):
# #     ATM_str = f"D_ATM"
# #     OTM_str = f"D_OTM_{round(OTM_moneyness * 100)}%"
# #     sgy_name = f"D_butterfly_spread_{round(OTM_moneyness * 100)}%"
# #     sgy_name_simple = ("D_butterfly_spread")
# #
# #     for low_high in ["low", "high"]:
# #         ATM_cost = df[f"{low_high}_put_{ATM_str}_price"] + df[f"{low_high}_call_{ATM_str}_price"]
# #         OTM_cost = df[f"{low_high}_put_{OTM_str}_price"] + df[f"{low_high}_call_{OTM_str}_price"]
# #         df[f"const_free_{low_high}_{sgy_name}"] = df[f"free_{low_high}_{sgy_name}"] / (ATM_cost - OTM_cost)
# #
# #     # 30 day version
# #     T1 = df["low days"].shift(1)
# #     T2 = df["high days"].shift(1)
# #
# #     low_high = "low"
# #     price_low  = df[f"{low_high}_put_{ATM_str}_price"] + df[f"{low_high}_call_{ATM_str}_price"] + df[f"{low_high}_put_{OTM_str}_price"] + df[f"{low_high}_call_{OTM_str}_price"]
# #     low_high = "high"
# #     price_high  = df[f"{low_high}_put_{ATM_str}_price"] + df[f"{low_high}_call_{ATM_str}_price"] + df[f"{low_high}_put_{OTM_str}_price"] + df[f"{low_high}_call_{OTM_str}_price"]
# #     df[f"const_30_{sgy_name_simple}"] = df[f"30_{sgy_name_simple}"] / T_day_interpolation(T1=T1, T2=T2, r1=price_low, r2=price_high)
# #     return df
#
# def add_delta_straddle_sgy(df):
#     add_delta_strangle_sgy(df) #Passed without 'OTM_moneyness'
#     return df
#
